In [1]:
import torch
from torch import nn
from torch.nn import init
import torch.utils.data as data_utils
from torch.autograd import Variable
import numpy as np
import heapq

In [2]:
# training data
dataset = np.loadtxt("./ml-1m/ratings.dat",delimiter='::',dtype=int)[:,[0,1,3]]
N_USERS = np.max(dataset[:,0])
N_ITEMS = np.max(dataset[:,1])
n_negatives = 4  ## 1正例对应n个负例 ##
users_items = np.zeros((N_USERS+1, N_ITEMS+1), dtype = np.int8)  # 混淆矩阵
user_input, item_input, labels = [],[],[]  # x1 x2 -> y
for u in range(dataset.shape[0]):   # 评分数据集隐式化
    users_items[dataset[u][0], dataset[u][1]] = 1
uipositives = list() # 作为测试集的交互正例
for i in range(N_USERS+1):
    if i==0: 
        continue
    uitems = dataset[dataset[:,0]==i]
    onepos = uitems[uitems[:,-1]==np.max(uitems),:2][0]
    uipositives.append(onepos)
    users_items[onepos[0], onepos[1]]=0
for uno, uitems in enumerate(users_items):
    if uno == 0:
        continue
    positives = np.nonzero(uitems)[0]
    n_sample = len(positives) * n_negatives
    negative_items = list(set(range(N_ITEMS+1))^set(positives))
    negatives = np.random.choice(negative_items, n_sample)  # 负采样 -- 不放回
    for i in range(len(positives)): # 正实例
        user_input.append(uno)
        item_input.append(positives[i])
        labels.append(1)
    for j in range(n_sample): # 负实例
        user_input.append(uno)
        item_input.append(negatives[j])
        labels.append(0)

In [3]:
# test data
utest = list()
itest = list()
for ui in uipositives:
    u = ui[0]
    i = ui[1]
    positives = np.nonzero(users_items[u])[0]
    negative_items = list(set(range(1,N_ITEMS+1))^set(positives))
    negatives_sample = np.random.choice(negative_items, 99)  # 负采样 -- 不放回
    negatives = [i]  # 正例
    for n in negatives_sample:
        negatives.append(n)  # 添加负例
    utest.append([u for j in range(100)])
    itest.append(negatives)
ytest = np.zeros((N_USERS,100))
ytest[:, 0] = 1

In [5]:
item_input

[1,
 150,
 260,
 527,
 531,
 588,
 594,
 595,
 608,
 661,
 720,
 745,
 783,
 914,
 919,
 938,
 1022,
 1028,
 1029,
 1035,
 1097,
 1193,
 1197,
 1207,
 1246,
 1270,
 1287,
 1545,
 1566,
 1721,
 1836,
 1907,
 1961,
 1962,
 2018,
 2028,
 2294,
 2321,
 2340,
 2355,
 2398,
 2687,
 2692,
 2762,
 2791,
 2797,
 2804,
 2918,
 3105,
 3114,
 3186,
 3408,
 3941,
 1784,
 2992,
 476,
 3779,
 1395,
 314,
 1884,
 286,
 2457,
 985,
 1996,
 1065,
 808,
 3733,
 1877,
 840,
 1925,
 164,
 502,
 1362,
 3185,
 2538,
 729,
 2366,
 2318,
 536,
 1244,
 187,
 3631,
 1877,
 946,
 1113,
 2432,
 3414,
 132,
 1708,
 1805,
 3415,
 219,
 602,
 328,
 1902,
 1667,
 779,
 1930,
 1884,
 2454,
 455,
 1379,
 3205,
 2323,
 2605,
 2978,
 2501,
 379,
 814,
 1930,
 110,
 187,
 1381,
 565,
 3598,
 453,
 3731,
 3194,
 82,
 2660,
 394,
 1539,
 89,
 2150,
 140,
 1295,
 487,
 3466,
 3299,
 2963,
 1991,
 3033,
 3793,
 3884,
 845,
 2616,
 3120,
 3617,
 3779,
 2629,
 2528,
 1431,
 1909,
 1405,
 878,
 644,
 1832,
 1929,
 1477,
 1597,
 1

In [4]:
# Hyper parameters
BATCH_SIZE = 256
LEARNING_RATE = 0.001
EPOCH = 12
USER_VECTOR_SIZE = 1    # len(one-hot of user vecter) 
ITEM_VECTOR_SIZE = 1    # len(one-hot of item vecter) 
LAYERS = [64, 32, 16, 8]   # MLP  0层为输入层  0层/2为嵌入层  
ACTIVATION = torch.relu

In [5]:
torch_x1 = torch.from_numpy(np.array(user_input, ndmin=2).T).type(torch.LongTensor)
torch_x2 = torch.from_numpy(np.array(item_input, ndmin=2).T).type(torch.LongTensor)
torch_y  = torch.from_numpy(np.array(labels, ndmin=2).T).type(torch.FloatTensor)

#x1 = Variable(torch.from_numpy(np.array(user_input, ndmin=2, dtype=np.float32).T))
#x2 = Variable(torch.from_numpy(np.array(item_input, ndmin=2, dtype=np.float32).T))
#y = Variable(torch.from_numpy(np.array(labels, ndmin=2, dtype=np.float32).T))
torch_dataset = data_utils.TensorDataset(torch_x1, torch_x2, torch_y)
loader = data_utils.DataLoader(dataset = torch_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

In [6]:
class Net(nn.Module):
    def __init__(self, user_vector_size, item_vector_size, layers,  
                 n_users, n_items, activation = torch.relu, batch_normalization = False, n_output = 1):
        super(Net, self).__init__()
        self.activation = activation
        self.do_bn = batch_normalization
        self.fcs = []
        self.bns = []
        self.n_layer  = len(layers)
        parameter_LeCun = np.sqrt(layers[-1])

        #self.bn_userInput = nn.BatchNorm1d(1)   # for input data
        #self.bn_itemInput = nn.BatchNorm1d(1)   # for input data
        
        self.mlp_user_embedding_layer = nn.Embedding(n_users+1, int(layers[0]/2))
        self._set_normalInit(self.mlp_user_embedding_layer, hasBias = False) 
        self.mlp_item_embedding_layer = nn.Embedding(n_items+1, int(layers[0]/2))
        self._set_normalInit(self.mlp_item_embedding_layer, hasBias = False) 
        
        #self.bn_user_elayer = nn.BatchNorm1d(mlp_embedding_size) 
        #self.bn_item_elayer = nn.BatchNorm1d(mlp_embedding_size)     
        
        for i in range(1, self.n_layer):               # build hidden layers and BN layers
            #input_size = layers[0] if i == 0 else layers[mlp_n_layers-i]
            fc = nn.Linear(layers[i-1], layers[i])
            self._set_normalInit(fc)                  # parameters initialization
            setattr(self, 'fc%i' % i, fc)       # IMPORTANT set layer to the Module
            self.fcs.append(fc)
            if self.do_bn:
                bn = nn.BatchNorm1d(layers[i])
                setattr(self, 'bn%i' % i, bn)   # IMPORTANT set layer to the Module
                self.bns.append(bn)

        self.predict = nn.Linear(layers[-1], n_output)         # output layer
        self._set_uniformInit(self.predict, parameter = parameter_LeCun)            # parameters initialization
        return

    def _set_normalInit(self, layer, parameter = [0.0, 0.01], hasBias=True):
        init.normal_(layer.weight, mean = parameter[0], std = parameter[1])
        if hasBias:
            init.normal_(layer.bias, mean = parameter[0], std = parameter[1])
        return
    
    def _set_uniformInit(self, layer, parameter = 5, hasBias = True):
        init.uniform_(layer.weight, a = - parameter, b = parameter)
        if hasBias:
            init.uniform_(layer.bias, a = - parameter, b = parameter)
        return
    
    def _set_heNormalInit(self, layer, hasBias=True):
        init.kaiming_normal_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.kaiming_normal_(layer.bias, nonlinearity='relu')
        return
    
    def _set_heUniformInit(self, layer, hasBias=True):
        init.kaiming_uniform_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.kaiming_uniform_(layer.bias, nonlinearity='relu')
        return

    def forward(self, x1, x2):
        #if self.do_bn: 
            #x1 = self.bn_userInput(x1)     # input batch normalization
            #x2 = self.bn_itemInput(x2)
        x1 = self.mlp_user_embedding_layer(x1)
        x2 = self.mlp_item_embedding_layer(x2)
        x3 = torch.cat((x1, x2), dim=1)
        x  = torch.flatten(x3, start_dim=1)
        for i in range(1, self.n_layer):
            x = self.fcs[i-1](x)
            if self.do_bn: 
                x = self.bns[i-1](x)   # batch normalization
            x = self.activation(x)
        out = torch.sigmoid(self.predict(x))
        return out

In [7]:
net = Net(user_vector_size = USER_VECTOR_SIZE, item_vector_size = ITEM_VECTOR_SIZE,
          layers = LAYERS, n_users = N_USERS, n_items = N_ITEMS, activation = ACTIVATION, batch_normalization = False, n_output = 1)
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)
loss_func = torch.nn.BCELoss()
print(net)

Net(
  (mlp_user_embedding_layer): Embedding(6041, 32)
  (mlp_item_embedding_layer): Embedding(3953, 32)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (predict): Linear(in_features=8, out_features=1, bias=True)
)


In [8]:
def getHitRatio(ranklist, gtItem):
    #HR击中率，如果topk中有正例ID即认为正确
    if gtItem in ranklist:
        return 1
    return 0

def getNDCG(ranklist, gtItem):
    #NDCG归一化折损累计增益
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return np.log(2) / np.log(i+2)
    return 0


In [9]:
def movieEval(model, loss_func, utest, itest, ytest, topK = 10):
    if len(utest)==len(itest)==len(ytest):
        n_users = len(utest)
    else:
        print('the length of test sets are not equal.')
        return
    hit = 0
    undcg = 0
    test_loss = list()
    for i in range(n_users):
        map_item_score = dict()
        x1test = Variable(torch.from_numpy(np.array(utest[i], ndmin=2).T).type(torch.LongTensor))
        x2test = Variable(torch.from_numpy(np.array(itest[i], ndmin=2).T).type(torch.LongTensor))
        y  = Variable(torch.from_numpy(np.array(ytest[i], ndmin=2).T).type(torch.FloatTensor))
        prediction = model(x1test, x2test)
        loss = loss_func(prediction, y)
        test_loss.append(loss.item())
        pred_vector = prediction.data.numpy().T[0]
        positive_item = itest[i][0]  # 取正例
        for j in range(len(itest[i])):
            map_item_score[itest[i][j]] = pred_vector[j]
        ranklist = heapq.nlargest(topK, map_item_score, key=map_item_score.get)
        hit += getHitRatio(ranklist, positive_item)
        undcg += getNDCG(ranklist, positive_item)
    mean_test_loss = np.mean(test_loss)
    hr = hit / n_users
    ndcg = undcg / n_users
    print('test_loss:', mean_test_loss)
    print('HR@', topK, ' = %.4f' % hr)
    print('NDCG@', topK, ' = %.4f' % ndcg)
    return mean_test_loss, hr, ndcg

In [ ]:
train_loss_list = list()
test_loss_list  = list()
hr_list = list()
ndcg_list = list()
for e in range(EPOCH):
    train_loss = list()
    for step, (batch_x1, batch_x2, batch_y) in enumerate(loader):
        x1, x2, y = Variable(batch_x1), Variable(batch_x2), Variable(batch_y)
        optimizer.zero_grad()
        prediction = net(x1, x2)
        loss = loss_func(prediction, y)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    print('------第'+str(e+1)+'个epoch------')
    mean_train_loss = np.mean(train_loss)
    print('train_loss:', mean_train_loss)
    train_loss_list.append(mean_train_loss)    
    test_loss, hr, ndcg = movieEval(net, loss_func, utest, itest, ytest)
    test_loss_list.append(test_loss)
    hr_list.append(hr)
    ndcg_list.append(ndcg)

------第1个epoch------
train_loss: 0.35027285884670206
test_loss: 0.1786077437441278
HR@ 10  = 0.4995
NDCG@ 10  = 0.2776
------第2个epoch------
train_loss: 0.3094524699672389
test_loss: 0.17772831390951052
HR@ 10  = 0.5689
NDCG@ 10  = 0.3214
------第3个epoch------
train_loss: 0.29028863589207315
test_loss: 0.16597943011985858
HR@ 10  = 0.6026
NDCG@ 10  = 0.3387
------第4个epoch------
train_loss: 0.27962927136580573
test_loss: 0.14710468863343684
HR@ 10  = 0.6232
NDCG@ 10  = 0.3495
